In [39]:
import numpy as np
from itertools import product
from qutip import Qobj, tensor, qeye, sigmax, sigmay, sigmaz, spre, choi_to_chi, choi_to_kraus

In [7]:
from forest.benchmarking.operator_tools import pauli_liouville2kraus

In [10]:
# Dimension 
d = 2

# Qubits
num_q = int(np.log2(d))

# Pauli matrices
P = [qeye(2), sigmax(), sigmay(), sigmaz()]

# Create all tensor products
sigma = list(map(tensor, map(list, product(P, repeat=num_q))))

In [37]:
# Create a amplitude damping channel
p = 0.2
ptm = np.array(
        [[1,0,0,0],
         [0,np.sqrt(1-p),0,0],
         [0,0,np.sqrt(1-p),0],
         [p,0,0,1-p]])
ptm

array([[1.        , 0.        , 0.        , 0.        ],
       [0.        , 0.89442719, 0.        , 0.        ],
       [0.        , 0.        , 0.89442719, 0.        ],
       [0.2       , 0.        , 0.        , 0.8       ]])

In [53]:
# Convert PTM to Choi
choi = 2 / d**2 * sum((ptm[i, j] * tensor(sigma[j].trans(), sigma[i])) 
                      for i in range(d**2) for j in range(d**2))
choi.dims = [[[2]*num_q, [2]*num_q], [[2]*num_q, [2]*num_q]]

In [54]:
# Convert Choi to Chi
chi = choi_to_chi(choi)

In [89]:
kraus = pauli_liouville2kraus(ptm)
sum(np.conj(k.T)@k for k in kraus)

array([[1.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j]])

In [152]:
# Convert Chi to PTM the fast way
A = np.zeros((d**2,d**2,d**2,d**2),dtype='complex128')
for i in range(d**2):
    for j in range(d**2):
        for k in range(d**2):
            for l in range(d**2):
                A[i,k,j,l] = (sigma[i]*sigma[k]*sigma[j]*sigma[l]).tr()

In [154]:
1/d**2*Qobj(np.einsum('lk,ikjl',chi.full(),A))

ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (4,4)->(4,4) (16,16,16,16)->(16,16,16,16) 

In [134]:
A_mat = np.reshape(A, (d**4,d**4))
chi_vec = np.reshape(chi, (d**4,1))
Qobj(1/d**2*np.reshape(A_mat@chi_vec, (d**2,d**2)))

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j -5.55111512e-17+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j  1.00000000e+00+0.00000000e+00j
   0.00000000e+00-5.55111512e-17j  0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+5.55111512e-17j
   1.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
 [-5.55111512e-17+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  1.00000000e+00+0.00000000e+00j]]

In [120]:
# Convert Chi to PTM
R = np.zeros((d**2,d**2))
for i in range(d**2):
    for j in range(d**2):
        R[j,i] = 1/d**2 * sum(chi[k,l]*(sigma[i]*sigma[k]*sigma[j]*sigma[l]).tr() for k in range(d**2) for l in range(d**2)).real

In [117]:
R[0,:]

array([ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00, -5.55111512e-17])

In [115]:
spre(tensor(sigmax(),sigmax()))

Quantum object: dims = [[[2, 2], [2, 2]], [[2, 2], [2, 2]]], shape = (16, 16), type = super, isherm = True
Qobj data =
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]